# Capstone Project: Ideal Places for an Asian Foodie in Toronto 


![](https://paladinsecurity.com/wp-content/uploads/2019/07/Toronto-Crop.png)

## Table of contents
* [Introduction](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## 1. Introduction <a name="introduction"></a>

### 1.1 Business Analysis

  ### Background
Most people have their own preference for food. Diet as an important componment of lifestyle can affect personal behaviours, decisions or even settlement. Canada as one of the world's most ethnically diverse and multicultural nations accepts many immigrations from other countries. Especially for Toronto,the provincial capital of Ontario and the most populous city in Canada, is an international centre of business, finance, arts, and culture, and is recognized as one of the most multicultural and cosmopolitan cities in the world. Immigrations may prefer their ethnic cuisine than others due to their personal perference, habituation and familiarity. Thus, studying the locations, density and quality of specific kind of cuisine is a crucial influence factor for people that can result in economic, social and cultral effect in Toronto neighborhood. 

  ### Objective
* In this project, we will try to find an optimal location for a immigration who prefers Asian food.
* Specifically, the report will be targetd to citizens interested in living closed to various **Asian restaurants** in **Toronto, Ontario, Canada**. 
* The result of this project can be used as a guidance for Asian citizens when they are looking for a residential location, planing for a restaurant business or taking meals with their families. 

# 2. Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:

* number of existing restaurants in the neighborhood (all restaurants)
* number of Asian restaurants in the neighborhood 
* density of Asian restaurants in the neighborhood 
* distance of neighborhood from city center

Following data sources will be needed to extract/generate the required information:

* centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using folium.Map
* number of restaurants and their type and location in every neighborhood will be obtained using Foursquare API
* coordinate of Toronto center will be obtained using folium

### 2.1 Neighborhood Candidates

Let's create latitude & longitude coordinates for centroids of Toronto. We will create a grid of cells covering our area of interest which is aprox. 12x12 killometers centered around Toronto city center.

#### import libraries

In [ ]:
!pip install folium
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
print('Install Done')

In [2]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitude_toronto = location.latitude
longitude_toronto = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_toronto, longitude_toronto))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [3]:
!pip install shapely
import shapely.geometry

!pip install pyproj
import pyproj

import math

In [4]:
def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

In [5]:
print('Coordinate transformation check')
print('-------------------------------')
print('Toronto center longitude={}, latitude={}'.format(longitude_toronto, latitude_toronto))
x, y = lonlat_to_xy(longitude_toronto, latitude_toronto)
print('Toronto center UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('Toronto center longitude={}, latitude={}'.format(lo, la))

Coordinate transformation check
-------------------------------
Toronto center longitude=-79.387207, latitude=43.653963
Toronto center UTM X=-5310408.823314988, Y=10507908.004696788
Toronto center longitude=-79.38720700000049, latitude=43.65396299999976


#### Let's create a hexagonal grid of cells: we offset every other row, and adjust vertical row spacing so that every cell center is equally distant from all it's neighbors.

In [6]:
toronto_center_x, toronto_center_y = lonlat_to_xy(longitude_toronto, latitude_toronto) # City center in Cartesian coordinates

k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min = toronto_center_x - 6000
x_step = 600
y_min = toronto_center_y - 6000 - (int(21/k)*k*600 - 12000)/2
y_step = 600 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 300 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(toronto_center_x, toronto_center_y, x, y)
        if (distance_from_center <= 6001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

            
import pickle
with open('lats_longs.pkl', 'wb') as f:
    pickle.dump((latitudes, longitudes), f)
        
print(len(latitudes), 'candidate neighborhood centers generated.')

364 candidate neighborhood centers generated.


#### View neighborhood candidates by using folium

In [7]:
map_toronto = folium.Map(location=[43.653963, -79.387207], zoom_start=13)
folium.Marker([43.653963, -79.387207], popup='Toronto').add_to(map_toronto)
for lat, lon in zip(latitudes, longitudes):
    #folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_toronto) 
    folium.Circle([lat, lon], radius=300, color='blue', fill=False).add_to(map_toronto)
    #folium.Marker([lat, lon]).add_to(map_toronto)
map_toronto

In [8]:
import pandas as pd

df_locations = pd.DataFrame({'Latitude': latitudes,
                             'Longitude': longitudes,
                             'X': xs,
                             'Y': ys,
                             'Distance from center': distances_from_center})

df_locations.head(10)

,Latitude,Longitude,X,Y,Distance from center
0,43.646694,-79.336638,-5.312209e+06,1.050219e+07,5992.495307
1,43.650419,-79.337200,-5.311609e+06,1.050219e+07,5840.376700
2,43.654145,-79.337761,-5.311009e+06,1.050219e+07,5747.173218
3,43.657871,-79.338323,-5.310409e+06,1.050219e+07,5715.767665
4,43.661597,-79.338885,-5.309809e+06,1.050219e+07,5747.173218
5,43.665323,-79.339447,-5.309209e+06,1.050219e+07,5840.376700
6,43.669050,-79.340009,-5.308609e+06,1.050219e+07,5992.495307
7,43.640753,-79.340238,-5.313109e+06,1.050271e+07,5855.766389
8,43.644478,-79.340800,-5.312509e+06,1.050271e+07,5604.462508
9,43.648203,-79.341362,-5.311909e+06,1.050271e+07,5408.326913


In [9]:
df_locations.to_pickle('./locations.pkl')    

### 2.2 Foursquare API get restaurants information
Now that we have our location candidates, let's use Foursquare API to get info on restaurants in each neighborhood.

We're interested in venues in 'food' category, but only those that are proper restaurants - Asian Restaurant. So we will include in out list only venues that have 'Asain restaurant' in category name. 

In [10]:
# Category IDs corresponding to Chinese restaurants were taken from Foursquare web site (https://developer.foursquare.com/docs/resources/categories):

food_category = '4d4b7105d754a06374d81259' # 'Root' category for all food-related venues

asian_restaurant_categories = ['52af3a5e3cf9994f4e043bea','52af3a723cf9994f4e043bec','52af3a7c3cf9994f4e043bed',
                               '58daa1558bbb0b01f18ec1d3','52af3a673cf9994f4e043beb','52af3a903cf9994f4e043bee',
                               '4bf58dd8d48988d1f5931735','52af3a9f3cf9994f4e043bef','52af3aaa3cf9994f4e043bf0',
                               '52af3ab53cf9994f4e043bf1','52af3abe3cf9994f4e043bf2','52af3ac83cf9994f4e043bf3',
                               '52af3ad23cf9994f4e043bf4','52af3add3cf9994f4e043bf5','52af3af23cf9994f4e043bf7',
                               '52af3ae63cf9994f4e043bf6','52af3afc3cf9994f4e043bf8','52af3b053cf9994f4e043bf9',
                               '52af3b213cf9994f4e043bfa','52af3b293cf9994f4e043bfb','52af3b343cf9994f4e043bfc',
                               '52af3b3b3cf9994f4e043bfd','52af3b463cf9994f4e043bfe','52af3b633cf9994f4e043c01',
                               '52af3b513cf9994f4e043bff','52af3b593cf9994f4e043c00','52af3b6e3cf9994f4e043c02',
                               '52af3b773cf9994f4e043c03','52af3b813cf9994f4e043c04','52af3b893cf9994f4e043c05',
                               '52af3b913cf9994f4e043c06','52af3b9a3cf9994f4e043c07','52af3ba23cf9994f4e043c08',
                               '4eb1bd1c3b7b55596b4a748f','52e81612bcbc57f1066b79fb','52af0bd33cf9994f4e043bdd',
                               '4deefc054765f83613cdba6f','52960eda3cf9994f4e043ac9','52960eda3cf9994f4e043acb',
                               '52960eda3cf9994f4e043aca','52960eda3cf9994f4e043acc','52960eda3cf9994f4e043ac7',
                               '52960eda3cf9994f4e043ac8','52960eda3cf9994f4e043ac5','52960eda3cf9994f4e043ac6',
                               '4bf58dd8d48988d111941735','55a59bace4b013909087cb0c','55a59bace4b013909087cb30',
                               '55a59bace4b013909087cb21','55a59bace4b013909087cb06','55a59bace4b013909087cb1b',
                               '55a59bace4b013909087cb1e','55a59bace4b013909087cb18','55a59bace4b013909087cb24',
                               '55a59bace4b013909087cb15','55a59bace4b013909087cb27','55a59bace4b013909087cb12',
                               '4bf58dd8d48988d1d2941735','55a59bace4b013909087cb2d','55a59a31e4b013909087cb00',
                               '55a59af1e4b013909087cb03','55a59bace4b013909087cb2a','55a59bace4b013909087cb0f',
                               '55a59bace4b013909087cb33','55a59bace4b013909087cb09','55a59bace4b013909087cb36',
                               '4bf58dd8d48988d113941735','56aa371be4b08b9a8d5734e4','56aa371be4b08b9a8d5734f0',
                               '56aa371be4b08b9a8d5734e7','56aa371be4b08b9a8d5734ed','56aa371be4b08b9a8d5734ea',
                               '4bf58dd8d48988d156941735','5ae9595eb77c77002c2f9f26','4eb1d5724b900d56c88a45fe',
                               '4bf58dd8d48988d1d1941735','56aa371be4b08b9a8d57350e','4bf58dd8d48988d149941735',
                               '56aa371be4b08b9a8d573502','52af39fb3cf9994f4e043be9','4bf58dd8d48988d14a941735']



#### Define methods to extreact infromation from results

In [11]:
import requests

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    address = address.replace(', Ontario', '')
    address = address.replace(', Canada', '')
    return address

def get_venues_near_location(lat, lon, category, client_id, client_secret):
    version = '20180928'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(client_id, client_secret, version, lat, lon, category, 500, 100)
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    venues = [(item['venue']['id'],
               item['venue']['name'],
               get_categories(item['venue']['categories']),
               (item['venue']['location']['lat'], item['venue']['location']['lng']),
               format_address(item['venue']['location']),
               item['venue']['location']['distance']) for item in results]
    return venues

#### Methods to collect restaurants info, we divided info into three categories(all restaurants, Asian restaurants, and restaurants in each location) 

In [12]:
# Let's now go over our neighborhood locations and get nearby restaurants; we'll also maintain a dictionary of all found restaurants and all found Asian restaurants

def get_restaurants(lats, lons):
    restaurants = {}
    asian_restaurants = {}
    location_restaurants = {}
    print('Obtaining venues around candidate locations:', end='')
    
    for lat, lon in zip(lats, lons):
        foursquare_client_id = 'FR4WQ0GHVC1KBPGN0I4WPGEGGDZJYGBZLHYFK3NSE431XOCS'
        foursquare_client_secret = 'J3G2DIGVREQHMFQ4NCF1B4UTERNSDXW4TXMNKLBG20MDKOSC'
        venues = get_venues_near_location(lat, lon, food_category, foursquare_client_id, foursquare_client_secret)
       
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
            restaurant = (venue_id, venue_name,venue_categories, venue_latlon[0], venue_latlon[1], venue_address, venue_distance, x, y)
            restaurants[venue_id] = restaurant
            print('*', end='')
            if is_asian(restaurant):
                asian_restaurants[venue_id] = restaurant
                location_restaurants.setdefault((lat, lon),[]).append(restaurant)
                print(' .', end='')
    print(' done.')
    return restaurants, asian_restaurants, location_restaurants

def is_asian (restaurant): 
    venue_category = restaurant[2]
    if venue_category[0][1] in asian_restaurant_categories:
        return True
    return False


#### Start to load results information 

In [13]:
# Try to load from local file system in case we did this before
with open('lats_longs.pkl','rb') as f:
    latitudes, longitudes = pickle.load(f)
    
restaurants = { }
asian_restaurants = { }
location_restaurants = { }

try:
    with open('restaurants.pkl','rb') as f:
        restaurants, asian_restaurants, location_restaurants = pickle.load(f)
except Exception as e:
    print(e)
    
    
if len(restaurants) == 0:
    restaurants, asian_restaurants, location_restaurants = get_restaurants(latitudes, longitudes)
    with open('restaurants.pkl','wb') as f:
        restaurants = pickle.dump((restaurants, asian_restaurants, location_restaurants), f)

In [14]:
import numpy as np

print('Total number of restaurants:', len(restaurants))
print('Total number of Asian restaurants:', len(asian_restaurants))
print('Percentage of Asian restaurants: {:.2f}%'.format(len(asian_restaurants) / len(restaurants) * 100))
print('Average number of restaurants in neighborhood:', np.array([len(r) for r in location_restaurants]).mean())

Total number of restaurants: 2272
Total number of Asian restaurants: 332
Percentage of Asian restaurants: 14.61%
Average number of restaurants in neighborhood: 2.0


In [15]:
print('List of all restaurants')
print('-----------------------')
for r in list(restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(restaurants))

List of all restaurants
-----------------------
('4f7f5bb87716da34d4ffb62d', 'Taste a licious fine foods catering', [('Café', '4bf58dd8d48988d16d941735')], 43.65411798360529, -79.33942079544067, 'Lakeshore, Toronto ON M4c 4v9', 300, -5310992.097965998, 10502384.429510107)
('4f6d1982e4b062083139cc8a', 'Village of Thai Peterborough', [('Thai Restaurant', '4bf58dd8d48988d149941735')], 43.654336, -79.339295, 'Peterborough', 295, -5310958.995644975, 10502366.095392086)
('4bb0eb64f964a520206a3ce3', 'Bar Rocket', [('Café', '4bf58dd8d48988d16d941735')], 43.64918, -79.342841, '225 Commissioners St., Toronto ON M4m 0a1', 268, -5311734.633907562, 10502865.738315133)
('4bc1df994cdfc9b6a3229521', "Gale's Snack Bar", [('Diner', '4bf58dd8d48988d147941735')], 43.658239, -79.339077, '539 Eastern Ave (Carlaw Ave), Toronto ON', 344, -5310340.672131851, 10502272.900043106)
('539a1eff498e1bf8518e4eb5', 'Food Dudes Pantry', [('Comfort Food Restaurant', '52e81612bcbc57f1066b7a00')], 43.65630014845779, -79.33

In [16]:
print('List of Asian restaurants')
print('---------------------------')
for r in list(asian_restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(asian_restaurants))

List of Asian restaurants
---------------------------
('4f6d1982e4b062083139cc8a', 'Village of Thai Peterborough', [('Thai Restaurant', '4bf58dd8d48988d149941735')], 43.654336, -79.339295, 'Peterborough', 295, -5310958.995644975, 10502366.095392086)
('5ba55b9ae0c0c9002c29d056', 'EAT BKK Thai Kitchen', [('Thai Restaurant', '4bf58dd8d48988d149941735')], 43.66045, -79.343113, '898 Queen St E (Logan Ave), Toronto ON M4M 1J3', 378, -5309937.7849121615, 10502700.609016394)
('5c0aa8ea9411f2002cb833d1', 'Juzz Sushi', [('Sushi Restaurant', '4bf58dd8d48988d1d2941735')], 43.660656, -79.3423, '912 Queen Street E (Queen and Logan), Toronto ON M4M 1J5', 441, -5309915.298535344, 10502603.102171617)
('4ada456ff964a520c22021e3', 'Hanoi 3 Seasons', [('Vietnamese Restaurant', '4bf58dd8d48988d14a941735')], 43.662199311171825, -79.33457571173595, '1135 Queen Street East (Larchmount Ave), Toronto ON', 353, -5309767.412248558, 10501683.947032444)
('4b18010ef964a5204bcb23e3', 'Sushi Marche', [('Sushi Restaura

In [17]:
print('Restaurants around location')
print('---------------------------')
for r in list(location_restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(location_restaurants))


Restaurants around location
---------------------------
[('4f6d1982e4b062083139cc8a', 'Village of Thai Peterborough', [('Thai Restaurant', '4bf58dd8d48988d149941735')], 43.654336, -79.339295, 'Peterborough', 467, -5310958.995644975, 10502366.095392086)]
[('4f6d1982e4b062083139cc8a', 'Village of Thai Peterborough', [('Thai Restaurant', '4bf58dd8d48988d149941735')], 43.654336, -79.339295, 'Peterborough', 125, -5310958.995644975, 10502366.095392086)]
[('5ba55b9ae0c0c9002c29d056', 'EAT BKK Thai Kitchen', [('Thai Restaurant', '4bf58dd8d48988d149941735')], 43.66045, -79.343113, '898 Queen St E (Logan Ave), Toronto ON M4M 1J3', 480, -5309937.7849121615, 10502700.609016394), ('5c0aa8ea9411f2002cb833d1', 'Juzz Sushi', [('Sushi Restaurant', '4bf58dd8d48988d1d2941735')], 43.660656, -79.3423, '912 Queen Street E (Queen and Logan), Toronto ON M4M 1J5', 445, -5309915.298535344, 10502603.102171617)]
[('4ada456ff964a520c22021e3', 'Hanoi 3 Seasons', [('Vietnamese Restaurant', '4bf58dd8d48988d14a941735'

## 3. Methodology <a name="methodology"></a>
In this project we will focus on the density of Asian restaurants in Toronto neighborhood. 

#### First step - Data Collection and Cleaning
Collected the required data (location and type)of every restaurants in Toronto.  

#### Second step - Data Analysis and Visualization
Our analysis will be calculation and exploration of '**restaurant location**' across different areas of toronto - we will use **red marker** to indicate their specific locations 

#### Third step - Data Analysis and Visualization
Our analysis will be calculation and exploration of '**restaurant density**' across different areas of toronto - we will use **heatmaps** to indicate a few promising areas close to center with much number of asian restaurants 


## 4. Analysis <a name="analysis"></a>

#### Let's now see all the collected restaurants in our area of interest on map, and let's also show Asian restaurants in red color.

In [18]:
map_toronto = folium.Map(location=[43.653963, -79.387207], zoom_start=13)
folium.Marker([43.653963, -79.387207], popup='Toronto').add_to(map_toronto)
for res in asian_restaurants.values():
    lat = res[3]; lon = res[4]
    color = 'red' 
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(map_toronto)
map_toronto

Let's perform some basic explanatory data analysis and derive some additional info from our raw data. First let's count the **number of restaurants in every area candidate**:

In [19]:
location_restaurants_count = np.array([len(r) for r in location_restaurants])
print(location_restaurants_count)
print('Average number of restaurants in every area with radius=300m:', np.array(location_restaurants_count).mean())

df_locations.head(10)
    


[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
Average number of restaurants in every area with radius=300m: 2.0


,Latitude,Longitude,X,Y,Distance from center
0,43.646694,-79.336638,-5.312209e+06,1.050219e+07,5992.495307
1,43.650419,-79.337200,-5.311609e+06,1.050219e+07,5840.376700
2,43.654145,-79.337761,-5.311009e+06,1.050219e+07,5747.173218
3,43.657871,-79.338323,-5.310409e+06,1.050219e+07,5715.767665
4,43.661597,-79.338885,-5.309809e+06,1.050219e+07,5747.173218
5,43.665323,-79.339447,-5.309209e+06,1.050219e+07,5840.376700
6,43.669050,-79.340009,-5.308609e+06,1.050219e+07,5992.495307
7,43.640753,-79.340238,-5.313109e+06,1.050271e+07,5855.766389
8,43.644478,-79.340800,-5.312509e+06,1.050271e+07,5604.462508
9,43.648203,-79.341362,-5.311909e+06,1.050271e+07,5408.326913


In [20]:
distances_to_asian_restaurant = []

for area_x, area_y in zip(xs, ys):
    min_distance = 10000
    for res in asian_restaurants.values():
        res_x = res[7]
        res_y = res[8]
        d = calc_xy_distance(area_x, area_y, res_x, res_y)
        if d<min_distance:
            min_distance = d
    distances_to_asian_restaurant.append(min_distance)

df_locations['Distance to Asian restaurant'] = distances_to_asian_restaurant

In [21]:
df_locations.head(10)

,Latitude,Longitude,X,Y,Distance from center,Distance to Asian restaurant
0,43.646694,-79.336638,-5.312209e+06,1.050219e+07,5992.495307,1261.862090
1,43.650419,-79.337200,-5.311609e+06,1.050219e+07,5840.376700,672.683232
2,43.654145,-79.337761,-5.311009e+06,1.050219e+07,5747.173218,180.857751
3,43.657871,-79.338323,-5.310409e+06,1.050219e+07,5715.767665,576.989014
4,43.661597,-79.338885,-5.309809e+06,1.050219e+07,5747.173218,385.105617
5,43.665323,-79.339447,-5.309209e+06,1.050219e+07,5840.376700,591.966883
6,43.669050,-79.340009,-5.308609e+06,1.050219e+07,5992.495307,267.246271
7,43.640753,-79.340238,-5.313109e+06,1.050271e+07,5855.766389,2177.454209
8,43.644478,-79.340800,-5.312509e+06,1.050271e+07,5604.462508,1587.927463
9,43.648203,-79.341362,-5.311909e+06,1.050271e+07,5408.326913,1010.801872


In [22]:
df_distance = df_locations.groupby(['X','Y']).mean().sort_values(by=['Distance to Asian restaurant']).head(60)
df_distance

,,Latitude,Longitude,Distance from center,Distance to Asian restaurant
X,Y,,,,
-5.311309e+06,1.050791e+07,43.648375,-79.386355,900.000000,25.130846
-5.309809e+06,1.051155e+07,43.655179,-79.418882,3686.461718,47.817035
-5.308909e+06,1.050687e+07,43.663991,-79.379738,1824.828759,57.057730
-5.308609e+06,1.050739e+07,43.665498,-79.384467,1873.499400,59.331327
-5.308309e+06,1.051103e+07,43.664853,-79.415868,3758.989226,62.005545
-5.309509e+06,1.050687e+07,43.660265,-79.379170,1374.772708,62.127251
-5.308309e+06,1.050999e+07,43.665571,-79.406978,2954.657341,62.756674
-5.311309e+06,1.050999e+07,43.646944,-79.404125,2264.950331,63.821705
-5.309809e+06,1.050739e+07,43.658046,-79.383331,793.725393,63.886499


In [23]:
print('Average distance to closest Asian restaurant from each area center:', df_locations['Distance to Asian restaurant'].mean())

Average distance to closest Asian restaurant from each area center: 832.7094865450824


**on average Asian restaurant can be found within ~900m** from every area center candidate. 

Let's crete a map showing **heatmap / density of restaurants** and try to extract some meaningfull info from that. Also, let's show **borders of Toronto boroughs** on our map and a few circles indicating distance of 1km, 2km and 3km.

In [24]:
restaurant_latlons = [[res[3], res[4]] for res in restaurants.values()]

asian_latlons = [[res[3], res[4]] for res in asian_restaurants.values()]

In [26]:
from folium import plugins
from folium.plugins import HeatMap
map_toronto = folium.Map(location=[43.653963, -79.387207], zoom_start=13)
folium.TileLayer('cartodbpositron').add_to(map_toronto) #cartodbpositron cartodbdark_matter
HeatMap(restaurant_latlons).add_to(map_toronto)
folium.Marker([43.653963, -79.387207]).add_to(map_toronto)
folium.Circle([43.653963, -79.387207], radius=1000).add_to(map_toronto)
folium.Circle([43.653963, -79.387207], radius=2000).add_to(map_toronto)
folium.Circle([43.653963, -79.387207], radius=3000).add_to(map_toronto)
map_toronto

In [27]:
map_toronto = folium.Map(location=[43.653963, -79.387207], zoom_start=13)
folium.TileLayer('cartodbpositron').add_to(map_toronto) #cartodbpositron cartodbdark_matter
HeatMap(asian_latlons).add_to(map_toronto)
folium.Marker([43.653963, -79.387207]).add_to(map_toronto)
folium.Circle([43.653963, -79.387207], radius=1000).add_to(map_toronto)
folium.Circle([43.653963, -79.387207], radius=2000).add_to(map_toronto)
folium.Circle([43.653963, -79.387207], radius=3000).add_to(map_toronto)

map_toronto

In [30]:
map_toronto = folium.Map(location=[43.653963, -79.387207], zoom_start=15)
folium.Marker([43.653963, -79.387207], popup='Toronto').add_to(map_toronto)
for res in asian_restaurants.values():
    lat = res[3]; lon = res[4]
    color = 'red' 
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(map_toronto)
HeatMap(asian_latlons).add_to(map_toronto)
map_toronto

## 5. Results and Discussion <a name="results"></a>

Our analysis shows that although there is a great number of restaurants in Toronto (~2000 in our initial area of interest which was 12x12km), there are high restaurant density closed to city center. Highest concentration of Asian restaurants was detected from Toronto center, so we focused our attention to areas esat-north, south-north , corresponding to Bloor-young and Bloor Street West of central Toronto. 

Result indicates that two zones containing largest number of Asian restaurant. This, of course, does not imply that those zones are actually optimal locations for taking Asian cuisines! Purpose of this analysis was to only provide infornation on Asian restaurants desnsity and their distant close to city center. Orther considerations, such as quality of service, proces, and rating, can be added into this project to further improve this project. 

## 6. Conclusion <a name="conclusion"></a>

Purpose of this project was to identify Toronto areas close to center with large number of restaurants (particularly Asian restaurants) in order to provide suggestions for an Asian citizen in Toronto who prefers national cuisines. By calculating restaurant density distribution from Foursquare data we have first identified general boroughs that justify further analysis, and then generated extensive collection of locations which satisfy some basic requirements regarding existing nearby restaurants. Clustering of those locations was then performed in order to create major zones of interest (containing greatest number of potential locations) and addresses of those zone centers met relevant conditions.
